In [1]:
import torch
from torchvision import datasets
import syft as sy

In [2]:
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


d:\program files\python\python37\tutorial-env\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
d:\program files\python\python37\tutorial-env\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


In [16]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 251a53b257a6444989011efcc1708231>,[flatten],flattened,<class 'torch.Tensor'>
1,<UID: 2c7334f392354cc2a34451806a5aa3c6>,[flatten_val],flattened validation data,<class 'torch.Tensor'>


In [4]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


d:\program files\python\python37\tutorial-env\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
d:\program files\python\python37\tutorial-env\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


In [17]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 74f66c05dd054507af41ccc17fcd42cc>,[flatten],flattened,<class 'torch.Tensor'>
1,<UID: e322bc917ad74396a231e66331b111bf>,[flatten_val],flattened validation data,<class 'torch.Tensor'>


In [18]:
labels = datasets.MNIST("mnist", download=True, train=True).targets[:10_000]  # don't need all to demonstrate value
labels_val = datasets.MNIST("mnist", download=True, train=False).targets[:5_000]

print(labels.shape, labels_val.shape)

torch.Size([10000]) torch.Size([5000])


In [19]:
train1 = duet1.store["flatten"]
val1 = duet1.store["flatten_val"]

train2 = duet2.store["flatten"]
val2 = duet2.store["flatten_val"]

In [20]:
dl_1 = duet1.torch.utils.data.DataLoader(train1, batch_size=128)
dl_1val = duet1.torch.utils.data.DataLoader(val1, batch_size=128)

dl_2 = duet2.torch.utils.data.DataLoader(train2, batch_size=128)
dl_2val = duet2.torch.utils.data.DataLoader(val2, batch_size=128)

dl_local = torch.utils.data.DataLoader(labels, batch_size=128)
dl_localval = torch.utils.data.DataLoader(labels_val, batch_size=128)

In [21]:
# In order to serialise our model we need to define it as below
from torch import nn

hidden_sizes = [128, 500]
output_size = 10

class SyNet_client(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet_client, self).__init__(torch_ref=torch_ref)
        self.lin = self.torch_ref.nn.Linear(392,64)
        
    def forward(self, x):
        x = self.lin(x)
        x = self.torch_ref.nn.functional.relu(x)
        return x
    
class SyNet_server(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet_server, self).__init__(torch_ref=torch_ref)
        self.lin2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.lin3 = nn.Linear(hidden_sizes[1], output_size)
        self.sft = nn.LogSoftmax(dim=1) 
        
    def forward(self, x):
        x = self.lin2(x)
        x = nn.ReLU()(x)
        x = self.lin3(x)
        x = self.sft(x)
        return x

In [22]:
#Model 1
model1 = SyNet_client(torch)
model1_ptr = model1.send(duet1)
opt1 = duet1.torch.optim.SGD(params=model1_ptr.parameters(),lr=0.01)

#Model 2
model2 = SyNet_client(torch)
model2_ptr = model2.send(duet2)
opt2 = duet2.torch.optim.SGD(params=model2_ptr.parameters(),lr=0.01)

#Model 3
model3 = SyNet_server(torch)
local_opt = torch.optim.SGD(params=model3.parameters(),lr=0.1)

In [24]:
for epoch in range(4):
    train_correct = 0
    train_total = 0
    
    for in1, in2, label in zip(dl_1, dl_2, dl_local):
        opt1.zero_grad()
        opt2.zero_grad()
        local_opt.zero_grad()

        a1_ptr = model1_ptr(in1)
        a1 = a1_ptr.clone().get(request_block=True)

        a2_ptr = model2_ptr(in2)
        a2 = a2_ptr.clone().get(request_block=True)
        
        a = torch.cat((a1, a2), 1)
        y = model3(a)
        
        criterion = nn.NLLLoss()
        loss = criterion(y, label.long())
        
        train_correct += y.max(1)[1].eq(label).sum().item()
        train_total += y.shape[0]
        print(loss)
        
        loss.backward()

        g1_ptr = a1.grad.clone().send(duet1)
        g2_ptr = a2.grad.clone().send(duet2)
        
        a1_ptr.backward(g1_ptr)
        a2_ptr.backward(g2_ptr)
        
        local_opt.step()
        opt1.step()
        opt2.step()
        
    correct = 0
    total = 0
    for in_val1, in_val2, label_val in zip(dl_1val, dl_2val, dl_localval):
        a1_ptr = model1_ptr(in_val1)
        a1 = a1_ptr.clone().get(request_block=True)

        a2_ptr = model2_ptr(in_val2)
        a2 = a2_ptr.clone().get(request_block=True)

        a = torch.cat((a1, a2), 1)
        y = model3(a)

        correct += y.max(1)[1].eq(label_val).sum().item()
        total += y.shape[0]
    print(f"Train Accuracy: {100*train_correct/train_total:.3f}%")
    print(f"Val Accuracy: {100*correct/total:.3f}%")

[2021-12-15T17:40:46.304663+0800][CRITICAL][logger]][22560] Request to access data length rejected.
[2021-12-15T17:40:46.304663+0800][CRITICAL][logger]][22560] Request to access data length rejected.


ValueError: Request to access data length rejected.